# Query to StreamCI

In [ ]:
URL = "https://api.streamci.org/query"
HEADERS = {"Content-Type": "application/json"}
AUTH_INFO = {
    "target": "ci4fairtest",
    "authtype": "secret",
    "secret_key": "<SECRET_KEY>"
}

In [ ]:
import requests
import json

payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {},
        # "project": ["deviceID"],
        "sort": "deviceID"
        
    }
}

headers = {"Content-Type": "application/json"}
response = requests.post(URL, json=payload, headers=HEADERS)
ndjson_text = response.text
lines = ndjson_text.strip().splitlines()
json_array = [json.loads(line) for line in lines]
print("Responses: ", len(json_array))

## Data Analysis with the Results

In [ ]:
import json
import pandas as pd


# Flatten and convert to DataFrame
df = pd.DataFrame(json_array)
df["time"] = pd.to_datetime(df["time"].apply(lambda x: x["$date"]))

# Group by deviceID and compute summary stats
summary = df.groupby("deviceID")[["val1", "val2"]].describe()
print(summary)

In [ ]:
import matplotlib.pyplot as plt

# Plot dots for each sensor
plt.figure(figsize=(10, 5))
for sensor in df["deviceID"].unique():
    subset = df[df["deviceID"] == sensor]
    plt.scatter(subset["time"], subset["val2"], label=sensor, s=10)

plt.xlabel("Time (US/Eastern)")
plt.ylabel("val2")
plt.title("val2 Over Time by Sensor")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()